In [138]:
import numpy as np
import pandas as pd
import pickle
import re
import os
import math


In [2]:
# Set model name here, options = {'iMM904', 'iYO844', 'iML1515'}
model_name = 'iYO844'


## Load data


In [102]:
model = pd.read_csv('../../data/models/{}_Model.csv'.format(model_name))
if model_name == 'iMM904':
    model.rename(columns={'Kcat': 'kcat'}, inplace=True)
    model.rename(columns={'Km': 'km'}, inplace=True)
if model_name == 'iYO844':
    model.rename(columns={'BSU': 'Rule'}, inplace=True)
if model_name == 'iML1515':
    model.rename(columns={'meta_name': 'reac_meta_name'}, inplace=True)

mvec = pd.read_csv('../../data/models/{}_mvec.csv'.format(model_name))
if model_name == 'iML1515':
    mvec.rename(columns={'meta_name': 'reac_meta_name'}, inplace=True)


In [105]:
# Reaction
# Drop useless columns
model = model.loc[:, ['Rule', 'reac_meta_name', 'reac_meta_value', 'Gibbs', 'kcat', 'km']]
model


,Rule,reac_meta_name,reac_meta_value,Gibbs,kcat,km
0,BSU00090,imp_c,-1.0,-244.30493,NaN,0.0243
1,BSU00090,xmp_c,1.0,-291.6249,NaN,NaN
2,BSU00140,dcmp_c,1.0,-206.3048,NaN,NaN
3,BSU00140,dcyt_c,-1.0,17.714563,NaN,NaN
4,BSU00140,adn_c,-1.0,87.03457,NaN,NaN
...,...,...,...,...,...,...
1516,BSU40420,dcamp_c,1.0,-315.34628,NaN,NaN
1517,BSU40420,imp_c,-1.0,-244.30493,NaN,NaN
1518,BSU40850,accoa_c,-1.0,-506.8639,NaN,NaN
1519,BSU40850,acmalt_c,1.0,-167.7031,NaN,NaN


In [106]:
# Molecule
mvec = mvec.drop(columns=['SMILES'])
mvec


,reac_meta_name,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,istnt_c,0.034616,0.052344,-0.032405,0.053052,-0.000082,-0.009388,-0.003238,-0.019588,0.006186,...,-0.007310,-0.001641,0.032719,-0.041696,0.001186,-0.010220,-0.033718,0.046453,0.007658,0.020313
1,hmbil_c,0.064757,0.106059,-0.085903,0.128846,0.018197,-0.014582,-0.001312,-0.049914,0.016280,...,-0.026731,0.008829,0.069371,-0.088491,0.002651,-0.033185,-0.076106,0.087244,0.015458,0.031300
2,tag6p__D_c,0.078863,0.114675,-0.105692,0.158943,0.023062,-0.020216,0.002675,-0.065022,0.022953,...,-0.025510,0.011606,0.070363,-0.117186,-0.005045,-0.047840,-0.076804,0.101711,0.004547,0.045300
3,fruur_c,0.068635,0.094918,-0.089442,0.126414,0.008797,-0.012519,0.002537,-0.058893,0.017126,...,-0.021080,0.003307,0.066820,-0.096962,-0.007166,-0.044819,-0.071273,0.095626,0.008698,0.032080
4,g3pg_c,0.066607,0.091234,-0.090395,0.127275,0.014320,-0.011735,0.003785,-0.055166,0.028846,...,-0.017062,0.004865,0.060276,-0.098263,-0.004310,-0.029819,-0.063236,0.089087,0.005903,0.026193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,cd2_c,0.013493,0.033151,-0.028466,0.033066,0.009564,-0.006915,0.000438,-0.017476,0.007908,...,-0.006782,0.005464,0.019208,-0.022484,0.005527,-0.017676,-0.027556,0.025720,-0.003598,0.017230
615,cd2_e,0.018030,0.024222,-0.023714,0.042989,-0.003234,-0.008184,0.000997,-0.021330,0.014891,...,-0.006587,-0.003747,0.018136,-0.027669,0.006030,-0.018318,-0.025210,0.024778,0.010029,0.010861
616,ca2_c,0.013637,0.032990,-0.035173,0.045935,0.000999,-0.009780,-0.007473,-0.013344,0.004615,...,-0.013903,0.008049,0.021022,-0.032126,-0.003476,-0.014989,-0.027117,0.037786,-0.004036,0.015706
617,ca2_e,0.016642,0.029947,-0.027672,0.050270,0.000152,-0.007983,-0.000066,-0.019465,0.002859,...,-0.007730,0.005879,0.031387,-0.032237,-0.005092,-0.014778,-0.022467,0.027048,0.002118,0.005958


## Node Feature


#### Enzyme feature


In [117]:
af2_output_dict = dict()

for root, dirs, files in os.walk('../../data/af2/bsu_pca/'):
    for file in files:
        if file.endswith(".pkl"):
            data = pickle.load(open(f'../../data/af2/bsu_pca/{file}', 'rb'))
            enzyme_name = file.split('.')[0]
            af2_output_dict[enzyme_name] = data

print(len(af2_output_dict))


827


In [122]:
# e_feature_dict = dict()

# for enzyme in af2_output_dict:
#     ori_logits = af2_output_dict[enzyme]['logits']
#     ori_single = af2_output_dict[enzyme]['single']

#     padded_logits = np.pad(ori_logits,
#                            ((0, 1600-ori_logits.shape[0]), (0, 1600-ori_logits.shape[0])), 'constant')
#     padded_single = np.pad(
#         ori_single, ((0, 1600-ori_single.shape[0])), 'constant')

#     result = {'logits': padded_logits, 'single': padded_single}
#     e_feature_dict[enzyme] = result

# with open('../../data/af2/bsu_pca/{}.pkl'.format(model_name), 'wb') as f:
#     pickle.dump(e_feature_dict, f)


In [123]:
e_feature_dict = pickle.load(open('../../data/af2/bsu_pca/{}.pkl'.format(model_name), 'rb'))
e_feature_dict['BSU37490'].keys()

dict_keys(['logits', 'single'])

In [124]:
# # Enzyme
# af2_output = pickle.load(
#     open('../../data/af2_out/{}_output.pkl'.format(model_name), 'rb'))

# # Rename the keys
# e_feature_dict = dict()
# for key in af2_output:
#     enzyme_name = key.split('/')[-1].split('.')[0]
#     e_feature_dict[enzyme_name] = af2_output[key]

# print(len(e_feature_dict))


#### Molecule feature

m_feature_dict = {reac_meta_name: list of features}


In [125]:
model[model['reac_meta_name'] == 'mettrna_c']


,Rule,reac_meta_name,reac_meta_value,Gibbs,kcat,km
21,BSU00380,mettrna_c,1.0,NaN,NaN,NaN
567,BSU15730,mettrna_c,-1.0,NaN,NaN,NaN


In [126]:
Gibbs = model.loc[:, ['reac_meta_name', 'Gibbs']].drop_duplicates(
    subset='reac_meta_name', keep='first')
Gibbs.set_index('reac_meta_name', inplace=True)
Gibbs['Gibbs'] = Gibbs['Gibbs'].apply(lambda x: re.sub("[^-.0-9]", "", str(x)))
Gibbs[Gibbs['Gibbs'] == '']



,Gibbs
reac_meta_name,
mettrna_c,
trnamet_c,
trdox_c,
trdrd_c,
2ahhmd_c,
...,...
ala_L_ser__L_c,
ser_L_ser__L_c,
gly_cys__L_c,


In [127]:
Gibbs['Gibbs'] = Gibbs['Gibbs'].apply(lambda x: float(x) if x != '' else 0.0)
Gibbs


,Gibbs
reac_meta_name,
imp_c,-244.304930
xmp_c,-291.624900
dcmp_c,-206.304800
dcyt_c,17.714563
adn_c,87.034570
...,...
msa_c,-81.378700
2pcpgc_c,-162.071960
quc_c,-101.981970


In [128]:
def convert_feature_to_list(x):
    reac_meta_name = x['reac_meta_name']
    gibbs_value = [float(Gibbs.loc[reac_meta_name, 'Gibbs'])]
    return gibbs_value + list(x['0':'127'])


mvec['feature'] = mvec.apply(convert_feature_to_list, axis=1)

m_feature_dict_t = mvec.loc[:, ['reac_meta_name', 'feature']].set_index(
    'reac_meta_name').to_dict('index')
m_feature_dict = dict()
for k in m_feature_dict_t:
    m_feature_dict[k] = m_feature_dict_t[k]['feature']

print(len(m_feature_dict))


619


## Data Clean

Enzyme:

- Some enzymes in model do not have af2 output
- i.e. delete the rows in model that refer to enzyme without key in enzyme_feature_dict

Molecule:

- Some molecules in model do not have mvec infomation
- i.e. delete the rows in model that refer to molecules without info in mvec


In [129]:
# Check missing enzymes in af2 output
e_model = list(model['Rule'].unique())      # enzymes in model sheet
e_af2 = list(e_feature_dict.keys())    # enzymes in af2 output

enzymes = pd.DataFrame({'model': e_model}, columns=['model'])
enzymes['in_af2'] = enzymes['model'].apply(lambda x: x in e_af2)

no_af2_output = enzymes[enzymes['in_af2'] == False].model
print('count = {}'.format(len(no_af2_output)))
print(list(no_af2_output))


count = 2
['BSU02480', 'BSU31980']


In [130]:
# Delete the rows in model that refer to enzyme without af2 output
print('Before deleting, there are {} links in model.'.format(model.shape[0]))
model_new = model.loc[:]
model_new['in_af2_output'] = model_new['Rule'].apply(
    lambda x: x in e_feature_dict.keys())
model_new = model_new[model_new['in_af2_output'] == True]
print('After deleting, there are {} links in model.'.format(
    model_new.shape[0]))
model_new.head()


Before deleting, there are 1521 links in model.
After deleting, there are 1515 links in model.


,Rule,reac_meta_name,reac_meta_value,Gibbs,kcat,km,in_af2_output
0,BSU00090,imp_c,-1.0,-244.30493,NaN,0.0243,True
1,BSU00090,xmp_c,1.0,-291.6249,NaN,NaN,True
2,BSU00140,dcmp_c,1.0,-206.3048,NaN,NaN,True
3,BSU00140,dcyt_c,-1.0,17.714563,NaN,NaN,True
4,BSU00140,adn_c,-1.0,87.03457,NaN,NaN,True


In [131]:
# Delete the rows in model that refer to molecules without mvec infomation
print('Before deleting, numer of links in model: {}'.format(
    model_new.shape[0]))
model_moles = model_new.reac_meta_name.unique()     # molecules in model sheet
mvec_moles = mvec.reac_meta_name.unique()           # molecules in mvec

model_new['mol_in_mvec'] = model_new['reac_meta_name'].apply(
    lambda x: x in list(mvec_moles))
model_new = model_new[model_new['mol_in_mvec'] == True]
print('After deleting, numer of links in model: {}'.format(model_new.shape[0]))
model_new.head()


Before deleting, numer of links in model: 1515
After deleting, numer of links in model: 1458


,Rule,reac_meta_name,reac_meta_value,Gibbs,kcat,km,in_af2_output,mol_in_mvec
0,BSU00090,imp_c,-1.0,-244.30493,NaN,0.0243,True,True
1,BSU00090,xmp_c,1.0,-291.6249,NaN,NaN,True,True
2,BSU00140,dcmp_c,1.0,-206.3048,NaN,NaN,True,True
3,BSU00140,dcyt_c,-1.0,17.714563,NaN,NaN,True,True
4,BSU00140,adn_c,-1.0,87.03457,NaN,NaN,True,True


---

## Node


### enzyme


In [132]:
num_e = model_new['Rule'].unique().shape[0]
print('Number of nodes of node_type_id 0: {}'.format(num_e))

# node_id, node_name, node_type_id
node_e = pd.DataFrame({'node_id': np.arange(num_e), 'node_name': model_new['Rule'].unique(), 'node_type_id': np.zeros_like(model_new['Rule'].unique())},
                      columns=['node_id', 'node_name', 'node_type_id'])

# enzyme_feature
def convert_feature_to_list(x):
    return e_feature_dict[x]


node_e['node_feature'] = node_e['node_name'].apply(convert_feature_to_list)
feat_len_e = len(node_e.iloc[0, 3])
print('Length of the enzyme feature: {}'.format(feat_len_e))

node_e


Number of nodes of node_type_id 0: 384
Length of the enzyme feature: 2


,node_id,node_name,node_type_id,node_feature
0,0,BSU00090,0,"{'logits': [[19.581642, -15.330734, -12.3731, ..."
1,1,BSU00140,0,"{'logits': [[37.44548, -5.164447, -2.6643798, ..."
2,2,BSU00150,0,"{'logits': [[29.771925, -7.539098, -5.8310566,..."
3,3,BSU00180,0,"{'logits': [[48.52335, 8.085651, 7.698195, 6.7..."
4,4,BSU00270,0,"{'logits': [[14.981688, -19.190962, -10.923376..."
...,...,...,...,...
379,379,BSU40190,0,"{'logits': [[20.193998, -18.742842, -16.95475,..."
380,380,BSU40320,0,"{'logits': [[4.1331844, -13.998642, -9.914898,..."
381,381,BSU40340,0,"{'logits': [[17.642204, -13.6244755, -11.74872..."
382,382,BSU40420,0,"{'logits': [[13.764095, -18.00769, -11.966258,..."


### molecule


In [133]:
num_m = model_new['reac_meta_name'].unique().shape[0]
print('Number of nodes of node_type_id 1: {}'.format(num_m))

# node_id, node_name, node_type_id
node_m = pd.DataFrame({'node_id': np.arange(num_e, num_e+num_m),
                       'node_name': model_new['reac_meta_name'].unique(),
                       'node_type_id': np.ones_like(model_new['reac_meta_name'].unique())},
                      columns=['node_id', 'node_name', 'node_type_id'])

# mol_feature
def convert_feature_to_list(x):
    return m_feature_dict[x]


node_m['node_feature'] = node_m['node_name'].apply(convert_feature_to_list)
feat_len_m = len(node_m.iloc[0, 3])
print('Length of the molecule feature: {}'.format(feat_len_m))

node_m


Number of nodes of node_type_id 1: 616
Length of the molecule feature: 129


,node_id,node_name,node_type_id,node_feature
0,384,imp_c,1,"[-244.30493, 0.043830927, 0.07581868, -0.05839..."
1,385,xmp_c,1,"[-291.6249, 0.05696255, 0.089833654, -0.069855..."
2,386,dcmp_c,1,"[-206.3048, 0.047462117, 0.070710495, -0.04960..."
3,387,dcyt_c,1,"[17.714563, 0.044684604, 0.07995401, -0.070259..."
4,388,adn_c,1,"[87.03457, 0.038609814, 0.05554623, -0.0555064..."
...,...,...,...,...
611,995,msa_c,1,"[-81.3787, 0.026036164, 0.046005037, -0.039838..."
612,996,2pcpgc_c,1,"[-162.07196, 0.058406197, 0.10162693, -0.07970..."
613,997,quc_c,1,"[-101.98197, 0.059084572, 0.100022756, -0.0800..."
614,998,glcn__D_c,1,"[-157.02974, 0.04241948, 0.055048183, -0.05639..."


In [134]:
node = pd.concat((node_e.loc[:, ['node_id', 'node_name', 'node_type_id', 'node_feature']],
                  node_m.loc[:, ['node_id', 'node_name', 'node_type_id', 'node_feature']]), axis=0)

node

,node_id,node_name,node_type_id,node_feature
0,0,BSU00090,0,"{'logits': [[19.581642, -15.330734, -12.3731, ..."
1,1,BSU00140,0,"{'logits': [[37.44548, -5.164447, -2.6643798, ..."
2,2,BSU00150,0,"{'logits': [[29.771925, -7.539098, -5.8310566,..."
3,3,BSU00180,0,"{'logits': [[48.52335, 8.085651, 7.698195, 6.7..."
4,4,BSU00270,0,"{'logits': [[14.981688, -19.190962, -10.923376..."
...,...,...,...,...
611,995,msa_c,1,"[-81.3787, 0.026036164, 0.046005037, -0.039838..."
612,996,2pcpgc_c,1,"[-162.07196, 0.058406197, 0.10162693, -0.07970..."
613,997,quc_c,1,"[-101.98197, 0.059084572, 0.100022756, -0.0800..."
614,998,glcn__D_c,1,"[-157.02974, 0.04241948, 0.055048183, -0.05639..."


In [135]:
node.to_pickle('./{}/node.pkl'.format(model_name))

### node.dat


In [111]:
# def encode_feature(f, node_type):
#     if node_type == 'e':
#         temp = list(np.zeros(feat_len_m))
#         return f + temp
#     elif node_type == 'm':
#         temp = list(np.zeros(feat_len_e))
#         return temp + f


In [112]:
# node_e['node_feature'] = node_e['enzyme_feature'].apply(
#     lambda x: encode_feature(x, 'e'))
# node_m['node_feature'] = node_m['mol_feature'].apply(
#     lambda x: encode_feature(x, 'm'))

# node = pd.concat((node_e.loc[:, ['node_id', 'node_name', 'node_type_id', 'node_feature']],
#                   node_m.loc[:, ['node_id', 'node_name', 'node_type_id', 'node_feature']]), axis=0)
# # node['feat_len'] = node['node_feature'].apply(lambda x: len(x))
# node


In [114]:
# Get a dictionary where keys are the node_name and the values are node_id
id_dict_t = node.loc[:, ['node_name', 'node_id']
                     ].set_index('node_name').to_dict('index')
id_dict = dict()
for k in id_dict_t:
    id_dict[k] = id_dict_t[k]['node_id']

print(len(id_dict))


1000


## Link


In [136]:
# Get reaction relations
link = model_new.loc[:, ['Rule', 'reac_meta_name', 'reac_meta_value']]
link.columns = ['enzyme_name', 'mole_name', 'reac_value']

# Get the index of the enzyme and molecule
link['e_idx'] = link['enzyme_name'].apply(lambda x: id_dict[x])
link['m_idx'] = link['mole_name'].apply(lambda x: id_dict[x])

# edge_type_id
link['edge_type_id'] = link['reac_value'].apply(lambda x: 1 if x > 0 else 0)

# node_id_source
link['node_id_source'] = link.apply(
    lambda x: x['e_idx'] if x['edge_type_id'] == 1 else x['m_idx'], axis=1)

# node_id_target
link['node_id_target'] = link.apply(
    lambda x: x['m_idx'] if x['edge_type_id'] == 1 else x['e_idx'], axis=1)

# edge_weight
link['edge_weight'] = link['reac_value'].apply(lambda x: abs(x))

link


,enzyme_name,mole_name,reac_value,e_idx,m_idx,edge_type_id,node_id_source,node_id_target,edge_weight
0,BSU00090,imp_c,-1.0,0,384,0,384,0,1.0
1,BSU00090,xmp_c,1.0,0,385,1,0,385,1.0
2,BSU00140,dcmp_c,1.0,1,386,1,1,386,1.0
3,BSU00140,dcyt_c,-1.0,1,387,0,387,1,1.0
4,BSU00140,adn_c,-1.0,1,388,0,388,1,1.0
...,...,...,...,...,...,...,...,...,...
1516,BSU40420,dcamp_c,1.0,382,514,1,382,514,1.0
1517,BSU40420,imp_c,-1.0,382,384,0,384,382,1.0
1518,BSU40850,accoa_c,-1.0,383,408,0,408,383,1.0
1519,BSU40850,acmalt_c,1.0,383,999,1,383,999,1.0


### link.dat


In [142]:
# Reorganize the dataframe
link = link.loc[:, ['node_id_source',
                    'node_id_target', 'edge_type_id', 'edge_weight']]
link.sort_values(by=['edge_type_id', 'node_id_source'], inplace=True)
link


,node_id_source,node_id_target,edge_type_id,edge_weight
0,384,0,0,1.0
208,384,60,0,1.0
1517,384,382,0,1.0
184,385,54,0,1.0
256,385,64,0,1.0
...,...,...,...,...
1510,380,921,1,1.0
1512,381,632,1,1.0
1513,381,435,1,1.0
1516,382,514,1,1.0


## Label


In [143]:
label = model_new.loc[:, ['Rule', 'km', 'kcat']]
label.columns = ['node_name', 'km', 'kcat']


def check_nan(x):
    return (not pd.isnull(x['km'])) or (not pd.isnull(x['kcat']))


label = label[label.apply(check_nan, axis=1)]

# node_id
label['node_id'] = label['node_name'].apply(lambda x: id_dict[x])

# node_type_id
label['node_type_id'] = np.zeros(label.shape[0], dtype=int)

# node_label


def get_combined_label(x):
    km = x['km']

    if pd.isnull(x['km']):
        km = 0.0

    return km


label['node_label'] = label.apply(get_combined_label, axis=1)
label = label.loc[:, ['node_id', 'node_type_id', 'node_label']]
label


,node_id,node_type_id,node_label
0,0,0,0.02430
8,2,0,0.00000
121,31,0,30.00000
125,33,0,0.28000
136,37,0,0.00000
...,...,...,...
1494,375,0,0.12000
1495,375,0,0.06000
1497,375,0,0.06000
1501,376,0,0.00428


In [144]:
label = model_new.loc[:, ['Rule', 'km', 'kcat']]
label.columns = ['node_name', 'km', 'kcat']


def check_nan(x):
    return (not pd.isnull(x['km'])) or (not pd.isnull(x['kcat']))


label = label[label.apply(check_nan, axis=1)]

# node_id
label['node_id'] = label['node_name'].apply(lambda x: id_dict[x])

# node_type_id
label['node_type_id'] = np.zeros(label.shape[0], dtype=int)

# node_label


def get_combined_label(x):
    km = x['km']
    kcat = 0.0  # math.log(x['kcat'])

    if pd.isnull(x['km']):
        km = 0.0
    if pd.isnull(x['kcat']):
        kcat = 0.0
    return [km, kcat]


label['node_label'] = label.apply(get_combined_label, axis=1)
label = label.loc[:, ['node_id', 'node_type_id', 'node_label']]
label


,node_id,node_type_id,node_label
0,0,0,"[0.0243, 0.0]"
8,2,0,"[0.0, 0.0]"
121,31,0,"[30.0, 0.0]"
125,33,0,"[0.28, 0.0]"
136,37,0,"[0.0, 0.0]"
...,...,...,...
1494,375,0,"[0.12, 0.0]"
1495,375,0,"[0.06, 0.0]"
1497,375,0,"[0.06, 0.0]"
1501,376,0,"[0.00428, 0.0]"


Use the mean of the km/kcat as the enzyme label


In [145]:
nodes_labels = {}

pre_repeated_node_id = -1
for r in range(label.shape[0]):
    cur_node_id = label.iloc[r, 0]
    if cur_node_id not in nodes_labels:
        nodes_labels[cur_node_id] = [label.iloc[r, 2]]
    else:
        nodes_labels[cur_node_id] += [label.iloc[r, 2]]
nodes_labels


{0: [[0.0243, 0.0]],
 2: [[0.0, 0.0]],
 31: [[30.0, 0.0]],
 33: [[0.28, 0.0]],
 37: [[0.0, 0.0]],
 38: [[15.43, 0.0]],
 47: [[24.075, 0.0]],
 50: [[11.0, 0.0]],
 64: [[0.043, 0.0], [6.65, 0.0], [1.53, 0.0], [0.043, 0.0]],
 67: [[0.0, 0.0], [0.0, 0.0]],
 79: [[0.008, 0.0], [0.008, 0.0]],
 80: [[0.001, 0.0]],
 88: [[5.0, 0.0]],
 89: [[1.0, 0.0]],
 91: [[338.0, 0.0], [3.42, 0.0], [0.0, 0.0]],
 93: [[1.62, 0.0]],
 107: [[0.0, 0.0]],
 108: [[0.012, 0.0]],
 110: [[0.0089, 0.0], [0.0089, 0.0]],
 118: [[0.05, 0.0]],
 119: [[0.865, 0.0]],
 126: [[0.0, 0.0]],
 131: [[0.9, 0.0]],
 135: [[0.0, 0.0]],
 150: [[3.13, 0.0], [1.07, 0.0]],
 155: [[0.0, 0.0], [27.0, 0.0]],
 159: [[1.7, 0.0]],
 165: [[8.0, 0.0]],
 167: [[0.4288, 0.0]],
 171: [[8.7, 0.0]],
 178: [[0.0, 0.0]],
 184: [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]],
 185: [[2.02, 0.0]],
 189: [[0.77, 0.0]],
 191: [[2.0, 0.0]],
 193: [[0.059, 0.0]],
 195: [[0.033, 0.0]],
 203: [[0.023, 0.0], [0.0575, 0.0]],
 204: [[0.1071, 0.0]],
 207: [[1.0,

In [146]:
node_mean_labels = {}
for node_id in nodes_labels:
    if len(nodes_labels[node_id]) > 1:
        sum_km = 0.0
        sum_kcat = 0.0
        for r in range(len(nodes_labels[node_id])):
            sum_km += nodes_labels[node_id][r][0]
            sum_kcat += nodes_labels[node_id][r][1]
        node_label = [sum_km/len(nodes_labels[node_id]),
                      sum_kcat/len(nodes_labels[node_id])]
    else:
        node_label = nodes_labels[node_id][0]
    node_mean_labels[node_id] = node_label
node_mean_labels


{0: [0.0243, 0.0],
 2: [0.0, 0.0],
 31: [30.0, 0.0],
 33: [0.28, 0.0],
 37: [0.0, 0.0],
 38: [15.43, 0.0],
 47: [24.075, 0.0],
 50: [11.0, 0.0],
 64: [2.0665, 0.0],
 67: [0.0, 0.0],
 79: [0.008, 0.0],
 80: [0.001, 0.0],
 88: [5.0, 0.0],
 89: [1.0, 0.0],
 91: [113.80666666666667, 0.0],
 93: [1.62, 0.0],
 107: [0.0, 0.0],
 108: [0.012, 0.0],
 110: [0.0089, 0.0],
 118: [0.05, 0.0],
 119: [0.865, 0.0],
 126: [0.0, 0.0],
 131: [0.9, 0.0],
 135: [0.0, 0.0],
 150: [2.1, 0.0],
 155: [13.5, 0.0],
 159: [1.7, 0.0],
 165: [8.0, 0.0],
 167: [0.4288, 0.0],
 171: [8.7, 0.0],
 178: [0.0, 0.0],
 184: [0.0, 0.0],
 185: [2.02, 0.0],
 189: [0.77, 0.0],
 191: [2.0, 0.0],
 193: [0.059, 0.0],
 195: [0.033, 0.0],
 203: [0.04025, 0.0],
 204: [0.1071, 0.0],
 207: [1.0, 0.0],
 211: [5.0, 0.0],
 212: [0.114, 0.0],
 213: [0.0, 0.0],
 215: [0.31551, 0.0],
 222: [0.0, 0.0],
 230: [0.22599999999999998, 0.0],
 232: [0.14, 0.0],
 237: [0.0, 0.0],
 240: [3.0, 0.0],
 248: [0.0, 0.0],
 252: [0.00033, 0.0],
 262: [0.0, 0.

In [147]:
label_mean = pd.DataFrame.from_dict(
    node_mean_labels, orient='index', columns=['km', 'kcat'])
label_mean['node_type_id'] = np.zeros(label_mean.shape[0], dtype=int)
label_mean.reset_index(inplace=True)
label_mean.columns = ['node_id', 'km', 'kcat', 'node_type_id']
label_mean['node_label'] = label_mean['node_id'].apply(
    lambda x: node_mean_labels[x])
label_mean = label_mean.loc[:, ['node_id', 'node_type_id', 'node_label']]
label_mean


,node_id,node_type_id,node_label
0,0,0,"[0.0243, 0.0]"
1,2,0,"[0.0, 0.0]"
2,31,0,"[30.0, 0.0]"
3,33,0,"[0.28, 0.0]"
4,37,0,"[0.0, 0.0]"
...,...,...,...
73,373,0,"[40.0, 0.0]"
74,374,0,"[11.2, 0.0]"
75,375,0,"[2.5100000000000002, 0.0]"
76,376,0,"[0.00428, 0.0]"


### label.dat


In [148]:
# Reorganize label
label = label.loc[:, ['node_id', 'node_type_id', 'node_label']]
label


,node_id,node_type_id,node_label
0,0,0,"[0.0243, 0.0]"
8,2,0,"[0.0, 0.0]"
121,31,0,"[30.0, 0.0]"
125,33,0,"[0.28, 0.0]"
136,37,0,"[0.0, 0.0]"
...,...,...,...
1494,375,0,"[0.12, 0.0]"
1495,375,0,"[0.06, 0.0]"
1497,375,0,"[0.06, 0.0]"
1501,376,0,"[0.00428, 0.0]"


## Save


In [149]:
node['node_feature_to_save'] = node['node_feature'].apply(
    lambda x: str(x).split('[')[1].split(']')[0])
node.to_csv('{}/node.dat'.format(model_name), sep='\t', index=False, header=False,
            columns=['node_id', 'node_name', 'node_type_id', 'node_feature_to_save'])
node = node.loc[:, ['node_id', 'node_name', 'node_type_id', 'node_feature']]
node


,node_id,node_name,node_type_id,node_feature
0,0,BSU00090,0,"{'logits': [[19.581642, -15.330734, -12.3731, ..."
1,1,BSU00140,0,"{'logits': [[37.44548, -5.164447, -2.6643798, ..."
2,2,BSU00150,0,"{'logits': [[29.771925, -7.539098, -5.8310566,..."
3,3,BSU00180,0,"{'logits': [[48.52335, 8.085651, 7.698195, 6.7..."
4,4,BSU00270,0,"{'logits': [[14.981688, -19.190962, -10.923376..."
...,...,...,...,...
611,995,msa_c,1,"[-81.3787, 0.026036164, 0.046005037, -0.039838..."
612,996,2pcpgc_c,1,"[-162.07196, 0.058406197, 0.10162693, -0.07970..."
613,997,quc_c,1,"[-101.98197, 0.059084572, 0.100022756, -0.0800..."
614,998,glcn__D_c,1,"[-157.02974, 0.04241948, 0.055048183, -0.05639..."


In [150]:
link.to_csv('{}/link.dat'.format(model_name),
            sep='\t', index=False, header=False)
link


,node_id_source,node_id_target,edge_type_id,edge_weight
0,384,0,0,1.0
208,384,60,0,1.0
1517,384,382,0,1.0
184,385,54,0,1.0
256,385,64,0,1.0
...,...,...,...,...
1510,380,921,1,1.0
1512,381,632,1,1.0
1513,381,435,1,1.0
1516,382,514,1,1.0


In [151]:
label


,node_id,node_type_id,node_label
0,0,0,"[0.0243, 0.0]"
8,2,0,"[0.0, 0.0]"
121,31,0,"[30.0, 0.0]"
125,33,0,"[0.28, 0.0]"
136,37,0,"[0.0, 0.0]"
...,...,...,...
1494,375,0,"[0.12, 0.0]"
1495,375,0,"[0.06, 0.0]"
1497,375,0,"[0.06, 0.0]"
1501,376,0,"[0.00428, 0.0]"


In [152]:
from sklearn.model_selection import train_test_split

label_train, label_test = train_test_split(label_mean, test_size=0.3)


In [153]:
def save_label(label, file_name):
    label['node_label_to_save'] = label['node_label'].apply(
        lambda x: str(x).split('[')[1].split(']')[0])
    label.to_csv('{}/{}'.format(model_name, file_name), sep='\t', index=False,
                 header=False, columns=['node_id', 'node_type_id', 'node_label_to_save'])
    label = label.loc[:, ['node_id', 'node_type_id', 'node_label']]
    display(label)


In [154]:
# save_label(label_mean, 'label')
save_label(label_train, 'label.dat')
save_label(label_test, 'label.dat.test')


,node_id,node_type_id,node_label
29,171,0,"[8.7, 0.0]"
40,211,0,"[5.0, 0.0]"
64,324,0,"[3.45, 0.0]"
57,288,0,"[0.12, 0.0]"
21,126,0,"[0.0, 0.0]"
63,319,0,"[1.47, 0.0]"
60,302,0,"[0.0304, 0.0]"
56,283,0,"[0.2293, 0.0]"
41,212,0,"[0.114, 0.0]"
28,167,0,"[0.4288, 0.0]"


,node_id,node_type_id,node_label
7,50,0,"[11.0, 0.0]"
55,279,0,"[0.0, 0.0]"
70,352,0,"[0.0, 0.0]"
20,119,0,"[0.865, 0.0]"
26,159,0,"[1.7, 0.0]"
12,88,0,"[5.0, 0.0]"
4,37,0,"[0.0, 0.0]"
42,213,0,"[0.0, 0.0]"
10,79,0,"[0.008, 0.0]"
52,266,0,"[0.47075, 0.0]"
